In [1]:
import numpy as np
import pandas as pd

In [2]:
# Get catalog from SIMBAD with relevant data. 
#
# From SIMBAD database, do "Criteria query"
# http://simbad.u-strasbg.fr/simbad/sim-fsam
# -Specify "Output options":
#    -Output format: "ASCII (aligned, | separator)"
#    -Check boxes to include radial velocity/redshift, Morphological type, and angular size in the output.
# -Search "redshift <0.3 & maintype=G & dimmajor < 180 & mttype=E" (then Ir and Sb; N output = 100 or 1000 each)
# -Copy/paste result into Emacs. (textedit will garble the columns).
# -Save file. Make sure path/name below matches the saved file.

gc=[]
gc=pd.read_csv("./SIMBAD_z0.3_N1000.csv",'|', header=0, skipinitialspace=1)

gc.columns = ['Number', 'identifier', 'typ', 'coords', 'redshift', 'Mag U', 'Mag B', 'Mag V', 'Mag R', 'Mag I','spec type','Morph+Ref', 'Angsizes','bib','note']

# Separate morphology type from reference info
gc[['MorphType','dum1','dum2']] = gc['Morph+Ref'].str.rsplit(expand=True)

#Drop unneeded dummy columns or extra info from the Simbad output
gc.drop(['Number','spec type','Morph+Ref','dum1','dum2'], axis=1, inplace=True)

gc

/home/jon/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,identifier,typ,coords,redshift,Mag U,Mag B,Mag V,Mag R,Mag I,Angsizes,bib,note,MorphType
0,UGC 836,G,01 18 59.4210922213 +44 17 11.036445854,0.038417,~,16.0,~,~,~,0.463 0.417,8,0,E
1,UGC 2328,G,02 50 59.6687854776 +37 27 59.855205906,0.016642,~,13.8,~,~,~,0.757 0.636,17,0,E
2,2MASX J02464352+3648314,G,02 46 43.5001088789 +36 48 31.461754537,0.043730,~,~,~,~,~,0.213 0.183,3,0,E
3,UGC 746,G,01 11 35.9732259931 +49 07 15.985062828,0.018066,~,14.8,~,~,~,0.913 0.566,10,0,E
4,UGC 1562,G,02 04 22.3 +45 46 34,0.025110,~,15.0,~,~,~,1.548 0.831,3,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,ESO 205-31,G,06 10 51.017 -50 41 36.44,0.035620,~,14.96,~,13.29,~,0.530 0.424,8,0,Sb
2274,ESO 205-7,G,05 54 14.6680203730 -51 58 35.335011693,0.006694,~,15.26,~,13.68,~,0.390 0.328,21,0,Sb
2275,ESO 202-18,G,04 26 19.283 -49 13 58.59,0.043760,~,15.21,~,13.78,~,0.440 0.334,10,0,Sb
2276,ESO 247-12,G,02 55 26.601 -42 22 05.81,0.019007,~,16.15,~,15.49,~,0.320 0.134,10,0,Sb


In [3]:
# Separate coordinate info
gc[['RAhr','RAmin','RAsec','Decdeg','Decamin','Decasec']] = gc['coords'].str.rsplit(expand=True)

# Generate hex coordinates
gc['RA'] = gc['RAhr'].map(str) + ':' + gc['RAmin'].map(str) + ':' + gc['RAsec'].map(str)
gc['Dec'] = gc['Decdeg'].map(str) + ':' + gc['Decamin'].map(str) + ':' + gc['Decasec'].map(str)

# Drop unneeded components
gc.drop(['coords','RAhr','RAmin','RAsec','Decdeg','Decamin','Decasec'], axis=1, inplace=True)

# gc

In [4]:
# Split angular sizes into 2 separate maj/min columns
gc[['Ang_maj_amin','Ang_min_amin']] = gc['Angsizes'].str.rsplit(expand=True)
gc.drop(['Angsizes'], axis=1, inplace=True)

# gc

In [5]:
# from astropy.cosmology import WMAP9 as cosmo
# cosmo.age(0) 
from astropy.cosmology import Planck15
# Planck15.age(0)
# print(Planck15.__doc__)


dist=[]

#Calculate angular diameter distance
dist=Planck15.angular_diameter_distance(gc.redshift)
print(type(dist))

#Add angular diam distance to catalog
gc['ang_diam_dist']=dist.value.tolist()

# gc

<class 'astropy.units.quantity.Quantity'>


In [6]:
# These calculations will simulate what results students would get for H0/universe age if we make
# the simplifying assumption that all observed galaxies are the same size as the Milky Way (not 
# including noise from the student measurements)

from astropy.coordinates import Angle
from astropy import units as u

# Assign column from catalog to new variable and specify unit in arcmin
ang_maj=Angle(gc.Ang_maj_amin, u.arcmin)
# ang_maj

# Convert major axis angular size to radians
# alternate way:  theta_radian=ang_maj *np.pi/180/60
theta_radian=ang_maj*u.arcmin.to(u.rad)
# theta_radian

# Original assumption:
# Calculate estimated distance if we assume galaxy is 100,000 light years across
# l_MW=100000*u.lightyear

# From text exchange with Alice Shapley:
# --Galaxy sizes are usually defined using D_25 - the isophote at which the surface brightness in some band 
#   (like B-band) is 25 mag/arcsec^2
# --As observed from elsewhere, using the D_25 criterion, you would only be able to detect the MW's extent out to
#   ~67,000 light years, so it would make sense to use this distance instead.
# --She isn't 100% sure of the 67k ly limit and will help with a more detailed calculation later.
l_MW=67000*u.lightyear

# This is the distance estimate you would get for the galaxy if you assume it is the same size as the Milky Way.
est_dist_Mpc = l_MW.to(u.Mpc)/theta_radian
est_dist_Mpc

# Add estimated distance to catalog
gc['est_dist_mpc']=est_dist_Mpc.value.tolist()

# Calculate actual physical size of galaxy
length_ltyr=theta_radian*dist*u.Mpc.to(u.lightyear)
length_ltyr

# Add physical size of galaxy to catalog
gc['length_maj']=length_ltyr.value.tolist()

In [7]:
# Calculate velocity from redshift and add to catalog
#
# To do: Confirm with someone that this is the correct formula to use and we are applying it correctly.
# (from https://astronomy.swin.edu.au/cosmos/c/cosmological+redshift)
# 
# z ~ HD/v - 1
#
# where H is at the given redshift and D is the comoving distance at that redshift
#
comov = Planck15.comoving_distance(gc.redshift)
h_z = Planck15.H(gc.redshift)
#print(comov, h_z)

z = gc.redshift.values
cosmological_vel = np.array(h_z*comov/(1+z))
# print(type(cosmological_vel), cosmological_vel)
# print(z)


# This is some old stuff used as a sanity check to verify that the velocity calculation above 
# is returning something sensible.
#
# vel=2.998e5*gc.redshift
# vel
# rat = vel/cosmological_vel

#print(type(comov), type(h_z), type(z), type(cosmological_vel), type(vel), type(rat))

gc['vel_cos']=cosmological_vel.tolist()
# gc

In [8]:
# Generate decimal degree values for importing into glue/WWT

RA_decimal = Angle(gc.RA, u.hour).degree
RA_decimal


dec_decimal = Angle(gc.Dec, u.deg).degree
dec_decimal


gc['RA_deg']=RA_decimal.tolist()
gc['Dec_deg']=dec_decimal.tolist()

gc

,identifier,typ,redshift,Mag U,Mag B,Mag V,Mag R,Mag I,bib,note,...,RA,Dec,Ang_maj_amin,Ang_min_amin,ang_diam_dist,est_dist_mpc,length_maj,vel_cos,RA_deg,Dec_deg
0,UGC 836,G,0.038417,~,16.0,~,~,~,8,0,...,01:18:59.4210922213,+44:17:11.036445854,0.463,0.417,162.260978,152.525276,71276.616154,11193.057893,19.747588,44.286399
1,UGC 2328,G,0.016642,~,13.8,~,~,~,17,0,...,02:50:59.6687854776,+37:27:59.855205906,0.757,0.636,72.165414,93.288247,51829.495056,4926.676765,42.748620,37.466626
2,2MASX J02464352+3648314,G,0.043730,~,~,~,~,~,3,0,...,02:46:43.5001088789,+36:48:31.461754537,0.213,0.183,183.529333,331.545553,37088.313213,12692.732867,41.681250,36.808739
3,UGC 746,G,0.018066,~,14.8,~,~,~,10,0,...,01:11:35.9732259931,+49:07:15.985062828,0.913,0.566,78.204693,77.348524,67741.621132,5342.563570,17.899888,49.121107
4,UGC 1562,G,0.025110,~,15.0,~,~,~,3,0,...,02:04:22.3,+45:46:34,1.548,0.831,107.771620,45.619640,158280.480820,7387.058372,31.092917,45.776111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,ESO 205-31,G,0.035620,~,14.96,~,13.29,~,8,0,...,06:10:51.017,-50:41:36.44,0.530,0.424,150.953737,133.243779,75905.235322,10399.053862,92.712571,-50.693456
2274,ESO 205-7,G,0.006694,~,15.26,~,13.68,~,21,0,...,05:54:14.6680203730,-51:58:35.335011693,0.390,0.328,29.382486,181.074879,10871.892073,1996.577127,88.561117,-51.976482
2275,ESO 202-18,G,0.043760,~,15.21,~,13.78,~,10,0,...,04:26:19.283,-49:13:58.59,0.440,0.334,183.648649,160.498188,76664.164621,12701.169286,66.580346,-49.232942
2276,ESO 247-12,G,0.019007,~,16.15,~,15.49,~,10,0,...,02:55:26.601,-42:22:05.81,0.320,0.134,82.184011,220.685009,24951.077517,5616.908573,43.860837,-42.368281


In [9]:
# Write out catalog for import into glue
gc.to_csv("./SIMBAD_z0.1_N1000_est_dist.csv", index=False)

In [10]:
# Methodology for calculating new age estimates based on students' measured values of H0

H0_student = 60

newcosmo = Planck15.clone(name='Planck15 mod', H0=H0_student)
newcosmo.age(0)

<Quantity 15.57052286 Gyr>

In [11]:
# Look at data and fits

from astropy.modeling import models, fitting

def new_model():
    return models.Linear1D(intercept=0, fixed={'intercept':True})

# Pull separate dataframes for each morphology type with specific redshift range
gc_E = gc[((gc["MorphType"] == "E") & (gc["redshift"]<0.3) & (gc["redshift"]>0.01))]
gc_Sb = gc[((gc["MorphType"] == "Sb") & (gc["redshift"]<0.3) & (gc["redshift"]>0.01))]
gc_Ir = gc[((gc["MorphType"] == "Ir") & (gc["redshift"]<0.3) & (gc["redshift"]>0.01))]

# See what happens if we randomly choose just 50 galaxies each since it will likely not be realistic
# to make available data for thousands of galaxies.
gc_E = gc_E.sample(n=50, replace=True)
gc_Sb = gc_Sb.sample(n=50, replace=True)
gc_Ir = gc_Ir.sample(n=50, replace=True)
nd = 0 #decimal places
fit = fitting.LinearLSQFitter()

distances = gc_E["est_dist_mpc"]
velocities = gc_E["vel_cos"]
H0_E = round(fit(new_model(), distances, velocities).slope.value, nd)

distances = gc_Sb["est_dist_mpc"]
velocities = gc_Sb["vel_cos"]
H0_Sb = round(fit(new_model(), distances, velocities).slope.value, nd)

distances = gc_Ir["est_dist_mpc"]
velocities = gc_Ir["vel_cos"]
H0_Ir = round(fit(new_model(), distances, velocities).slope.value, nd)

In [12]:
import glue_jupyter as gj
from glue.config import viewer_tool
from glue.viewers.common.tool import Tool
import ipyvuetify as v

In [13]:
from glue.core import Data
from glue_jupyter.bqplot.scatter import BqplotScatterView

app = gj.jglue()
scatter_viewer = BqplotScatterView(session=app.session)

gc_data = [
    (gc_E, "E", "purple"),
    (gc_Sb, "Sb", "red"),
    (gc_Ir, "Ir", "blue")
]
scatter_data = []
for g, label, color in gc_data:
    data = Data(distance=g["est_dist_mpc"].values, velocity=g["vel_cos"].values, length=g["length_maj"], redshift=g["redshift"], label=label)
    app.add_data(data)
    for d in scatter_data:
        for field in ['distance', 'velocity', 'length', 'redshift']:
            app.add_link(data, field, d, field)
    scatter_viewer.add_layer(data)
    scatter_viewer.layers[-1].state.color = color
    scatter_data.append(data)
scatter_viewer.state.x_att = data.id['distance']
scatter_viewer.state.y_att = data.id['velocity']
scatter_viewer.show()

/home/jon/opt/anaconda3/envs/cds-nb-update/lib/python3.9/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')


LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:home': {'tooltip': '…

In [14]:
class DataSelectionMenu(v.Menu):
        
    def __init__(self, data_items, action):
        self.data_items = data_items
        self.items = [ v.ListItem(children=[v.ListItemTitle(children=[data.label])]) for data in self.data_items ]
        self.action = action
        
        for item in self.items:
            item.on_event('click', self._select)
        super().__init__(
            v_slots=[{
                'name': 'activator',
                'variable': 'menuData',
                'children': v.Btn(v_on="menuData.on", children=[
                    'menu'
                ])
            }],
            class_='ma-2',
            children=[v.List(children=self.items)]
        )
        
    def _select(self, widget, event, data):
        name = widget.children[0].children[0]
        for item in self.data_items:
            if name == item.label:
                self.action(item)
                self.close()
                return
        

In [15]:
from os.path import abspath, join
from IPython.display import display

@viewer_tool
class HubbleFitterTool(Tool):
    
    icon = "line_fit.svg"
    tool_id = "hubble_fitter"
    action_text = "Does Hubble fit"
    tool_tip = "Does Hubble fit"
    
    _fitted_x_att = "x"
    _fitted_y_att = "y_fit"
    
    def __init__(self, viewer):
        super(HubbleFitterTool, self).__init__(viewer)
        self.fitted_data = {}
        
    def activate(self):
        for data in self._data_in_viewer:
            self._do_fit(data)
        #action = lambda x: self._do_fit(x)
        #menu = DataSelectionMenu(self._data_in_viewer, action)
        #display(menu)
        
    def _do_fit(self, data):
        
        # Do the fit
        fitted = HubbleFitterTool._fit_line(data, self.viewer.state.x_att, self.viewer.state.y_att)
        
        # Remove a previous fit for this data, if there is one
        already_fitted = self.fitted_data.get(data.label, None)
        if already_fitted:
            self.viewer.remove_data(already_fitted)
            self._data_collection.remove(already_fitted)
            
        # Add the data
        # We add links
        #     original x <--> fitted x
        #     original y <--> fitted y
        self._data_collection.append(fitted)
        app = self._application
        app.add_link(data, self.viewer.state.x_att, fitted, HubbleFitterTool._fitted_x_att)
        app.add_link(data, self.viewer.state.y_att, fitted, HubbleFitterTool._fitted_y_att)
        self.viewer.add_data(fitted)
        self.fitted_data[data.label] = fitted
        
        layer = self._layer_for_data(data)
        if layer:
            self.viewer.layers[-1].state.color = layer.state.color
        
    def close(self):
        pass
    
    def _layer_for_data(self, data):
        return next((layer for layer in self.viewer.layers if layer.state.layer == data), None)
    
    @property
    def _data_in_viewer(self):
        return [ layer.state.layer for layer in self.viewer.layers ]
    
    @property
    def _application(self):
        return self.viewer.session.application
    
    @property
    def _data_collection(self):
        return self._application.data_collection
        
    @staticmethod
    def _fit_line(data, x_name, y_name):
        x_arr = data[x_name]
        y_arr = data[y_name]
        fit = fitting.LinearLSQFitter()
        line_init = models.Linear1D(intercept=0, fixed={'intercept':True})
        fitted_line = fit(line_init, x_arr, y_arr)
        fitted_name = "fitted_%s_vs_%s" % (y_name, x_name)
        kwargs = { HubbleFitterTool._fitted_x_att: x_arr, HubbleFitterTool._fitted_y_att: fitted_line(x_arr) }
        return Data(label="%s Fitted" % data.label, **kwargs)

In [16]:
# Just a scatter viewer with only our custom tool
class HubbleScatterViewer(BqplotScatterView):
    tools = ["bqplot:panzoom", HubbleFitterTool.tool_id]
    inherit_tools = False

In [17]:
hs_viewer = HubbleScatterViewer(session=app.session)
for data in scatter_data:
    hs_viewer.add_data(data)
hs_viewer.show()

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:panzoom': {'tooltip'…

In [18]:
hub_tool = hs_viewer.toolbar.tools['hubble_fitter']
hs_viewer.toolbar

BasicJupyterToolbar(tools_data={'bqplot:panzoom': {'tooltip': 'Interactively pan and zoom around', 'img': 'dat…

In [19]:
from glue_jupyter.bqplot.histogram import BqplotHistogramView

hist_viewer = BqplotHistogramView(session=app.session)
for data in scatter_data:
    hist_viewer.add_data(data)
    hist_viewer.figure_widget.marks[-1].opacities = [0.5]
hist_viewer.state.x_att = data.id['length']
hist_viewer.state.hist_n_bin = 30
hist_viewer.show()

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:home': {'tooltip': '…

In [20]:
gv_viewer = BqplotHistogramView(session=app.session)
gv_data = Data(velocity=gc["vel_cos"])
app.add_data(gv_data)
gv_viewer.add_data(gv_data)
gv_viewer.state.hist_n_bin = 50
gv_viewer.show()

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:home': {'tooltip': '…

In [21]:
# Look at peculiar velocities
# Data from https://ui.adsabs.harvard.edu/abs/2006Ap.....49..450K/abstract

pec_v=[]
pec_v=pd.read_csv("./2006_Karachentsev_pec_velocities.dat",'|', header=[2], skiprows=[3],skipinitialspace=1)

pec_v

/home/jon/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,2MFGC,GLON GLAT,Jmag,W50,HRV,V3K,Hr,Vd,Vpec,Vp
0,2,91.89 -65.99,12.15,389,6547,6196,6323,-43,-24,19
1,13,108.04 -40.87,12.43,422,6804,6453,6704,-125,-135,-10
2,16,102.30 -54.33,12.40,595,14747,14386,13452,-84,1402,1486
3,23,100.87 -57.05,13.60,246,6339,5979,5521,-75,537,612
4,31,110.45 -34.25,13.64,280,7610,7272,6663,-143,724,867
...,...,...,...,...,...,...,...,...,...,...
2719,17990,109.04 -34.36,13.44,282,7674,7334,7465,-141,13,154
2720,17998,107.80 -38.73,13.96,296,8074,7725,9328,-130,-1377,-1247
2721,18009,98.57 -58.50,12.05,402,6493,6133,7122,-69,-858,-788
2722,18012,93.70 -63.93,11.49,447,5781,5426,6426,-50,-893,-843


In [22]:
pv_data = Data(velocity=pec_v["Vpec"], label="PecV")
pv_viewer = BqplotHistogramView(session=app.session)
app.add_data(pv_data)
pv_viewer.add_data(pv_data)
pv_viewer.show()

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(tools_data={'bqplot:home': {'tooltip': '…